In [10]:
import pandas as pd
import json
import os




In [ ]:

output_dir = "data/data_outputs"  
utility_path = f"{output_dir}/utility_results.csv"
mia_path = f"{output_dir}/model_mia_auc.json"


In [ ]:
#
utility_df = pd.read_csv(utility_path)
with open(mia_path, "r") as f:
    model_mia_aucs = json.load(f)


In [ ]:
# FIlter for RF models only
rf_df = utility_df[utility_df["Model"] == "RandomForest"].copy()

# Match MIA AUC values by lowercasing model names
rf_df["Privacy (MIA AUC)"] = rf_df["Trained_On"].str.lower().map(model_mia_aucs)



In [ ]:
# Reorder and rename column names for summary table
summary_table = rf_df[["Trained_On", "Accuracy", "F1", "AUC", "Privacy (MIA AUC)"]]
summary_table = summary_table.rename(columns={
    "Trained_On": "Model",
    "AUC": "ROC AUC",
    "F1": "F1 Score"
})
summary_table = summary_table.reset_index(drop=True)



In [ ]:
# Create a separate MIA AUC table
mia_table = pd.DataFrame(model_mia_aucs.items(), columns=["Model", "MIA AUC"])
mia_table["Model"] = mia_table["Model"].str.upper()
mia_table = mia_table.sort_values(by="MIA AUC", ascending=False).reset_index(drop=True)


In [ ]:

print("\n--- Privacy–Utility Summary (Random Forest) ---")
print(summary_table.to_string(index=False))

print("\n--- Model-Based MIA AUCs ---")
print(mia_table.to_string(index=False))


--- Privacy–Utility Summary (Random Forest) ---
 Model  Accuracy  F1 Score  ROC AUC  Privacy (MIA AUC)
  Real  0.858617  0.684052 0.909621                NaN
 CTGAN  0.833313  0.612839 0.871170           0.519071
Copula  0.762191  0.031031 0.714579           0.502688
  TVAE  0.810220  0.616910 0.861700           0.498276

--- Model-Based MIA AUCs ---
 Model  MIA AUC
 CTGAN 0.519071
COPULA 0.502688
  TVAE 0.498276


In [ ]:
worst_case_path = os.path.join(output_dir, "worst_case_mia_auc.json")

# Load JSON
with open(worst_case_path, "r") as f:
    worst_case_aucs = json.load(f)

# Creat DF
df = pd.DataFrame(worst_case_aucs.items(), columns=["Model", "Worst-Case MIA AUC"])
df["Model"] = df["Model"].str.upper()
df = df.sort_values(by="Worst-Case MIA AUC", ascending=False).reset_index(drop=True)


print("\n--- Worst-Case Membership Inference AUCs ---")
print(df.to_string(index=False))



--- Worst-Case Membership Inference AUCs ---
 Model  Worst-Case MIA AUC
 CTGAN            0.555848
  TVAE            0.514356
COPULA            0.506916
